# Data module

## PWA data frame

In [ ]:
from pycompwa.data import create

In [ ]:
frame = create.empty_frame(
    particle_names=['gamma', 'pi0', 'pi0'],
    number_of_rows=3)
frame

In [ ]:
frame['gamma', 'p_x'] = [-0.520903, -0.285015, 0.632325]
frame['gamma', 'p_y'] = [0.885259, 0.520381, -0.779928]
frame['gamma', 'p_z'] = [0.655934, -0.996574, -0.892786]
frame['gamma', 'E'] = [1.21872, 1.15982, 1.34357]

frame['pi0-1', 'p_x'] = [0.653672, 0.452265, 0.113717]
frame['pi0-1', 'p_y'] = [-0.813022, -0.76188, 0.605441]
frame['pi0-1', 'p_z'] = [-1.01763, 0.00723327, 0.718613]
frame['pi0-1', 'E'] = [1.46359, 0.896256, 0.956093]

frame['pi0-2', 'p_x'] = [-0.132769, -0.16725, -0.746043]
frame['pi0-2', 'p_y'] = [-0.0722372, 0.241499, 0.174487]
frame['pi0-2', 'p_z'] = [0.361697, 0.989341, 0.174172]
frame['pi0-2', 'E'] = [0.414596, 1.04082, 0.797233]

In [ ]:
print('Particles:', frame.pwa.particles)
print('Momentum labels:', frame.pwa.momentum_labels)
print('Weights:', frame.pwa.weights)
print('Average pi0 mass:\n',
      frame[['pi0-1', 'pi0-2']].pwa.mass.stack().mean())
print('Average gamma 3-momentum:\n',
      frame['gamma'].pwa.rho.mean(), '+/-',
      frame['gamma'].pwa.rho.std())

We'll see more of these properties after we import some real data.

## Import and export data

In [ ]:
from pycompwa.data import io

In [ ]:
frame_data = io.pawian.read_hists_file('jpsi_f0_gammapipi.root', 'data')
frame_fit = io.pawian.read_hists_file('jpsi_f0_gammapipi.root', 'fit')
frame_fit

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
f0_data = frame_data['pi0_1'] + frame_data['pi0_2']
f0_fit = frame_fit['pi0_1'] + frame_fit['pi0_2']

f0_data.pwa.mass.hist(label='data',
                      bins=60, density=True, alpha=.5)
f0_fit.pwa.mass.hist(label='fit',
                     bins=60, density=True, histtype='step', color='red',
                     weights=frame_fit.pwa.intensities)

plt.xlabel('$M(\pi^0,\pi^0)$ [GeV]')
plt.legend()
plt.gca().set_title('f0 resonance')
plt.show()

In [ ]:
selection = frame_data[abs(f0_data.pwa.mass - 0.990) < 0.05]
io.pawian.write_ascii(selection, 'filtered_data.dat')
io.pawian.read_ascii('filtered_data.dat', ['gamma', 'pi0', 'pi0'])

## Conversion to kinematic variables

In [ ]:
import logging
import pycompwa.ui as pwa
# not interested in warnings now
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
pwa.Logging('ERROR');

In [ ]:
from pycompwa.expertsystem.ui.system_control import \
    StateTransitionManager, InteractionTypes

initial_state = [("J/psi", [-1, 1])]
final_state = [("gamma"), ("pi0"), ("pi0")]
tbd_manager = StateTransitionManager(initial_state, final_state,
                                     formalism_type='helicity',
                                     topology_building='isobar')

tbd_manager.set_allowed_interaction_types([InteractionTypes.EM])
tbd_manager.allowed_intermediate_particles = ['f0(980)']

graph_interaction_settings_groups = tbd_manager.prepare_graphs()
solutions, _ = tbd_manager.find_solutions(
    graph_interaction_settings_groups)

from pycompwa.expertsystem.amplitude.helicitydecay import \
    HelicityAmplitudeGeneratorXML

model_file = 'jpsi_f0_gammapipi.xml'
xml_generator = HelicityAmplitudeGeneratorXML()
xml_generator.generate(solutions)
xml_generator.write_to_file(model_file)

In [ ]:
from pycompwa.data import convert

In [ ]:
try:
    convert.pandas_to_events(frame_data, model_file)
except Exception as exc:
    print('EXCEPTION:', exc)

In [ ]:
from pycompwa.data import naming

In [ ]:
naming.particle_to_id(frame_data, model_file)
frame_data.pwa.particles

As you can see, the $\gamma$ has been nicely renamed to its final state ID,
but the renaming failed for the pions (it would have worked if the separator
used for the added index for duplicate particles were a ``-``). If we follow
the second suggestion, it will work:

In [ ]:
mapping = {'gamma': 2, 'pi0_1': 3, 'pi0_2': 4}
frame_data.rename(columns=mapping, inplace=True)
frame_fit.rename(columns=mapping, inplace=True)
events_data = convert.pandas_to_events(frame_data, model_file)
events_fit = convert.pandas_to_events(frame_fit, model_file)

In [ ]:
from pycompwa.data import append
from pycompwa.data import kinematics

In [ ]:
set_data = kinematics.compute(events_data, model_file)
set_fit = kinematics.compute(events_fit, model_file)
naming.id_to_particle(frame_data, model_file, make_unique=True)
naming.id_to_particle(frame_fit, model_file, make_unique=True)
append(frame_data, convert.data_set_to_pandas(set_data))
append(frame_fit, convert.data_set_to_pandas(set_fit))
frame_data.pwa.other_columns

Finally, we can plot the distributions of the kinematic variables (as computed by ComPWA) **of the imported data**.

In [ ]:
var = 'theta_2_4_vs_3'
frame_data[var].hist(label='data',
                     bins=60, density=True, alpha=.5)
frame_fit[var].hist(label='fit',
                    bins=60, density=True, histtype='step', color='red',
                    weights=frame_fit.pwa.intensities)
plt.gca().set_title(naming.replace_ids(var, model_file))
plt.legend();

## Import and conversion in one go

In [ ]:
import pycompwa.ui as pwa
pwa.write_ascii_data(events_data, 'sample_data.dat')
pwa.write_root_data(events_fit, 'sample_fit.root')

In [ ]:
imported_events_data = pwa.read_ascii_data('sample_data.dat')
imported_events_fit = pwa.read_root_data('sample_fit.root')

We can now do the whole procedure above in one go with ``compute_kinemaitcs``:

In [ ]:
frame_data = convert.events_to_pandas(
    imported_events_data, model_file, compute_kinematics=True)
frame_fit = convert.events_to_pandas(
    imported_events_fit, model_file, compute_kinematics=True)

In [ ]:
print('Particles:', frame_data.pwa.particles)
print('Number of kinematic variables:', len(frame_fit.pwa.other_columns))
print('Average M(pi0, pi0)',
      frame_data['mSq_(3,4)'].mean(), '+/-',
      frame_data['mSq_(3,4)'].std())